<font color='darkblue' size=5>
NN non-linear with linear reg final layer+sampler
</font>

--------

- Err var = 0.25
- WITH beta sampling while generating rewards
- A small coefficient for X1X2 interaction
- small correlation btwn X1 and X2


In [24]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [25]:
import numpy as np
import pandas as pd
import torch
import pickle
import sklearn
import scipy
import scipy.stats as stats
from matplotlib import pyplot as plt
import seaborn as sns
import datetime
import yaml
import random
from tqdm.auto import tqdm

In [26]:
pd.options.display.float_format = "{:,.2f}".format
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',200)
pd.set_option('display.width',500)
sns.set()
tqdm.pandas()

In [27]:
import sklearn
from sklearn.linear_model import LinearRegression
import statsmodels as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

from scipy.stats import invgamma
import copy


In [28]:
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F

# Test create offline data 

## Assume mean and covariance matrices

In [29]:
mu_vec = np.array([[1   , 0.1 ,  1 ,0.01  ],
                   [1.25, 0.3 ,0.5 ,0.015 ],
                   [1.5 , 0.5 ,0.1 ,0.01  ],
                   [1.75, 0.3 ,0.5 ,0.015 ],
                   [2.0 , 0.1 ,  1 ,0.01  ] ])
# mu_vec = np.array([[1   , 0.1 ,  1 ,0.0 ],
#                    [1.25, 0.3 ,0.5 ,0.0 ],
#                    [1.5 , 0.5 ,0.1 ,0.0 ],
#                    [1.75, 0.3 ,0.5 ,0.0 ],
#                    [2.0 , 0.1 ,  1 ,0.0 ] ])

mu_vec

array([[1.   , 0.1  , 1.   , 0.01 ],
       [1.25 , 0.3  , 0.5  , 0.015],
       [1.5  , 0.5  , 0.1  , 0.01 ],
       [1.75 , 0.3  , 0.5  , 0.015],
       [2.   , 0.1  , 1.   , 0.01 ]])

In [30]:
num_arms   = 5
matrixSize = 4
parmCovMat = []

for i in range(num_arms):
    _tempA  = np.random.rand(1000, matrixSize)
    pCovMat = np.dot(_tempA.transpose(), _tempA) #To get a positive definite matrix
    pCovMat = pCovMat/2000 #To reduce the scale of variances and covariances. 
    pCovMat +=np.ones((matrixSize, matrixSize)) * np.array([[0.0, -0.1,-0.1, 0.0],
                                                            [-0.1, 0.0,-0.1, 0.0],
                                                            [-0.1,-0.1, 0.0, 0.0],
                                                            [ 0.0, 0.0, 0.0, 0.0]]) # To increase the variances [as in ridge regression] i.e. implying lower covariances between parameters
    pCovMat[matrixSize-1,:] = 0.0
    pCovMat[:,matrixSize-1] = 0.0
    pCovMat[matrixSize-1,matrixSize-1] = 0.005
    parmCovMat.append(pCovMat)

#     pCovMat +=np.ones((matrixSize, matrixSize)) * np.array([[0.2, -0.05, 0.05],
#                                                             [-0.05, 0.5, 0.0],
#                                                             [-0.05, 0.0, 0.5]]) # To increase the variances [as in ridge regression] i.e. implying lower covariances between parameters
parmCovMat

[array([[0.16944353, 0.02443279, 0.02225808, 0.        ],
        [0.02443279, 0.16564317, 0.02068194, 0.        ],
        [0.02225808, 0.02068194, 0.16085486, 0.        ],
        [0.        , 0.        , 0.        , 0.005     ]]),
 array([[0.16093093, 0.02212785, 0.02051755, 0.        ],
        [0.02212785, 0.16535366, 0.0216939 , 0.        ],
        [0.02051755, 0.0216939 , 0.16199029, 0.        ],
        [0.        , 0.        , 0.        , 0.005     ]]),
 array([[0.16363424, 0.01989482, 0.02782333, 0.        ],
        [0.01989482, 0.16154533, 0.02798823, 0.        ],
        [0.02782333, 0.02798823, 0.17400999, 0.        ],
        [0.        , 0.        , 0.        , 0.005     ]]),
 array([[0.16688277, 0.02488543, 0.02542759, 0.        ],
        [0.02488543, 0.16537696, 0.02370056, 0.        ],
        [0.02542759, 0.02370056, 0.16508331, 0.        ],
        [0.        , 0.        , 0.        , 0.005     ]]),
 array([[0.16482668, 0.02970371, 0.02577395, 0.        ],
      

## Create a batch of data. 

## Instead of fixed table, you could also wrap new data generation within a function

In [31]:
X_mumat  = np.array([5, 5])
X_covmat = np.array([[5, 1],
                      [1 ,5]]) #2 variables

datarows = 10000
X_matrix     = np.random.multivariate_normal(mean= X_mumat, cov= X_covmat, size=datarows, check_valid='warn', tol=1e-8)
sampled_data = np.hstack((np.ones([datarows,1]), X_matrix, np.expand_dims(X_matrix[:, 0] * X_matrix[:, 1], axis=1) ) )


In [32]:
sampled_data.shape

(10000, 4)

In [33]:
sampled_data, sampled_data.mean(axis=0)

(array([[ 1.        ,  5.56351044,  7.22094545, 40.17380542],
        [ 1.        ,  4.47257664,  1.00233506,  4.48302036],
        [ 1.        ,  2.21599463,  1.8523974 ,  4.1049027 ],
        ...,
        [ 1.        ,  5.69757285,  2.46158286, 14.02504765],
        [ 1.        ,  8.22817079,  2.64615415, 21.77300827],
        [ 1.        ,  5.0680296 ,  5.49379004, 27.84269052]]),
 array([ 1.        ,  5.01875685,  4.99680941, 26.1706049 ]))

In [34]:
class ColDataset(Dataset):
    def __init__(self, num_arms, dx, mu_vec, parmCovMat, knownfeatures):
        self.knownfeatures      = knownfeatures
        self.sampled_data       = np.array(dx, dtype=np.float32) 
        self.ERROR_VARIANCE_STD = 0.25
        self.mu_vec             = mu_vec
        self.parmCovMat         = parmCovMat
        self.num_actions        = num_arms
        if isinstance(self.knownfeatures, np.ndarray ):
             self.orgtraffic_known_features     = torch.from_numpy(self.sampled_data[:,self.knownfeatures]).float()
        else:
             self.orgtraffic_known_features     = torch.from_numpy(self.sampled_data                      ).float()
        
    def sampled_parm(self, act:int=0, size=1):
        # mu_vec, parmCovMat  
        return np.random.multivariate_normal(mean= mu_vec[act], cov= parmCovMat[act], size=size, check_valid='warn', tol=1e-8)
        #return mu_vec[act] #population coefficients, as if deterministic

    def yourORG_as_a_blackBox(self, d, act):
        parm    = self.sampled_parm(act=act, size=1)
        reward  = np.dot(parm , np.transpose(d)[..., np.newaxis]).ravel()
        err     = np.random.normal(loc=0, scale=self.ERROR_VARIANCE_STD, size=1)
        reward +=err
        return np.array(reward)#[..., np.newaxis]
    
    def __len__(self):
        return len(self.sampled_data)
    
    def __getitem__(self, idx):
        #call recommended action
        act  = np.random.choice(np.arange(self.num_actions), size=1)[0] #CHANGE  #self.lp.yourAPI_recommending_actions(mdlidx, context = orgtraffic_known_features, size=len(orgtraffic_known_features))
        d_np = self.sampled_data[idx]
        d    = self.orgtraffic_known_features[idx]
        #pass actions to your organization and measure reward 
        y    = torch.from_numpy(self.yourORG_as_a_blackBox(d_np, act)).float() 
        act  = torch.tensor(act, dtype=torch.long)
        #print(f"d - shape - {d.shape}  ; y.shape - ", y.shape)
        
        return d, act, y



In [35]:
class mdlnn(torch.nn.Module):
    def __init__(self, layersizes, hidden_drops, num_features, num_arms):
        super(mdlnn, self).__init__()
        
        self.num_arms     = num_arms
        self.hidden_drops = hidden_drops
        self.layersizes   = layersizes
        self.num_features = num_features - 1 #exclude intercept term because we use batchnorm and linear layers include bias coeff
        
        if (self.layersizes[0] ==1)&(len(self.layersizes)==1):
            llist = [torch.nn.Linear(self.num_features, 1, bias=True) for i in range(self.num_arms)]
            
            self.lins = torch.nn.ModuleList(llist)
        else:
            assert self.layersizes[0] >1, 'When training non-linear model, have atleast 2 nodes in 1st layer'
            assert self.layersizes[-1]>1, 'When training non-linear model, do not define final layers as part of this set, define them in output '
            llist = [torch.nn.Linear(self.num_features, layersizes[0], bias=True)] #shared layers for all arms
            for i in range(len(self.layersizes)-1): #shared layers for all arms
                  llist.append(torch.nn.Linear(layersizes[i], layersizes[i+1], bias=True))
                    
            self.lins = torch.nn.ModuleList(llist)
            # batch norm
            self.bns_input = torch.nn.BatchNorm1d(self.num_features)

            # dropouts
            self.drops     = torch.nn.ModuleList([torch.nn.Dropout(drop) for drop in hidden_drops])

            #activation
            self.actfunc   = torch.nn.LeakyReLU(0.01)
            self.output    = torch.nn.ModuleList([torch.nn.Linear(layersizes[-1], 1, bias=True) for i in range(self.num_arms)])
            for ll in self.output:
                torch.nn.init.xavier_normal_(ll.weight) #torch.nn.init.orthogonal_(ll.weight)
        
        for ll in self.lins:
           torch.nn.init.xavier_normal_(ll.weight) #torch.nn.init.orthogonal_(ll.weight) 
        
    def forward(self, x, act):
        if (self.layersizes[0] ==1)&(len(self.layersizes)==1):
            x_l = [l(x[act==i,:]) for i, l in enumerate(self.lins)]
        else:
            #print("entered in")
            x = self.bns_input(x)
            for i in range(len(self.layersizes)):
                x = self.lins[i](x)
                #x = self.actfunc(x)
                x = self.drops[i](x)
            print(x.shape, 'x')
            x_l = [l(x[act==i,:]) for i, l in enumerate(self.output)]
        return x_l

    def forward_custom(self, x):
        if 1==0:
            x_l = [l(x).detach().numpy() for i, l in enumerate(self.lins)]
            return x_l, x.detach().numpy()
        else:
            #print("entered in")
            x = self.bns_input(x)
            for i in range(len(self.layersizes)):
                x = self.lins[i](x)
                #x = self.actfunc(x)
                x = self.drops[i](x)
            
            x_l = [l(x).detach().numpy() for i, l in enumerate(self.output)]
            return x_l, x.detach().numpy() #THIS has last layer y_hat (for each arm), AND, L-1 layer representation (which is agnostic of the arm/action). Pass this L-1 as z/context in Linear-TS
            
        return x_l, np.array([])


In [36]:
torch.manual_seed(42)
np.random.seed(42)
KNOWNFEATURES = 0
train_ds = ColDataset(num_arms, sampled_data, mu_vec, parmCovMat, knownfeatures=KNOWNFEATURES) # train_trf_data[:, cat_vars_index], train_trf_data[:, continuous_vars_index], y1)
valid_ds = ColDataset(num_arms, sampled_data, mu_vec, parmCovMat, knownfeatures=KNOWNFEATURES)
print("Data Loaded and transformed")

trn_dataloader = DataLoader(train_ds, batch_size=4000, shuffle=True, num_workers=4)
val_dataloader = DataLoader(valid_ds, batch_size=10000000, shuffle=False, num_workers=4)

Data Loaded and transformed


In [37]:
# if isinstance(KNOWNFEATURES, np.ndarray ):
#      mdl = mdlnn(layersizes=[4], hidden_drops=[0.7], num_features=len(KNOWNFEATURES)   , num_arms=num_arms)
# else:



In [38]:
mdl = mdlnn(layersizes=[4], hidden_drops=[0.7], num_features=sampled_data.shape[1], num_arms=num_arms)
mdl

mdlnn(
  (lins): ModuleList(
    (0): Linear(in_features=3, out_features=4, bias=True)
  )
  (bns_input): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drops): ModuleList(
    (0): Dropout(p=0.7, inplace=False)
  )
  (actfunc): LeakyReLU(negative_slope=0.01)
  (output): ModuleList(
    (0): Linear(in_features=4, out_features=1, bias=True)
    (1): Linear(in_features=4, out_features=1, bias=True)
    (2): Linear(in_features=4, out_features=1, bias=True)
    (3): Linear(in_features=4, out_features=1, bias=True)
    (4): Linear(in_features=4, out_features=1, bias=True)
  )
)

In [40]:
optimizer = torch.optim.SGD(mdl.parameters(), lr=1e-3, momentum=0.9)
#optimizer = torch.optim.Adam(list(mdl.parameters()), lr = 1e-3)
lrscheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.95)
#lrscheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-6, max_lr=1e-3, mode='triangular') #mode='triangular2'

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

mseloss  = F.mse_loss

In [45]:
tr_loss = []
mdl.train()
train_offpolicy = []
for train_epoch in range(1, 100):
    biter = 0
    for dx,act,y in trn_dataloader:
        print(y.shape, 'y')
        print(act.shape)
        optimizer.zero_grad()
        pred_list      = mdl.forward(dx[:,1:], act)  #exclude intercept/constant
        print(len(pred_list), 'p')
        print(pred_list[0].shape)
        y_list         = [y[act==i,:] for i in range(num_arms)]
        print(len(y_list), 'yl')
        print(y_list[0].shape)
        loss = 0
        for i in range(num_arms):
            loss += mseloss(pred_list[i].flatten().to(DEVICE), y_list[i].flatten().to(DEVICE))
            #print(f'loss - after arm {i} loss is added - {loss}')
        loss.backward()
        optimizer.step()
        lrscheduler.step()
        
        biter +=1
        #print(f"train_epoch - {train_epoch} ; biter - {biter} ; loss -- {loss} ; dx - {dx.shape} ; y - {y.shape}")
        
        train_offpolicy.append([train_epoch, biter , dx.detach().numpy() , act.flatten().detach().numpy() , y.flatten().detach().numpy() ] )

        tr_loss.append([train_epoch, biter, loss.item()])



torch.Size([4000, 1]) y
torch.Size([4000])
torch.Size([4000, 4]) x
5 p
torch.Size([770, 1])
5 yl
torch.Size([770, 1])
torch.Size([4000, 1]) y
torch.Size([4000])
torch.Size([4000, 4]) x
5 p
torch.Size([770, 1])
5 yl
torch.Size([770, 1])
torch.Size([2000, 1]) y
torch.Size([2000])
torch.Size([2000, 4]) x
5 p
torch.Size([367, 1])
5 yl
torch.Size([367, 1])
torch.Size([4000, 1]) y
torch.Size([4000])
torch.Size([4000, 4]) x
5 p
torch.Size([770, 1])
5 yl
torch.Size([770, 1])
torch.Size([4000, 1]) y
torch.Size([4000])
torch.Size([4000, 4]) x
5 p
torch.Size([770, 1])
5 yl
torch.Size([770, 1])
torch.Size([2000, 1]) y
torch.Size([2000])
torch.Size([2000, 4]) x
5 p
torch.Size([367, 1])
5 yl
torch.Size([367, 1])
torch.Size([4000, 1]) y
torch.Size([4000])
torch.Size([4000, 4]) x
5 p
torch.Size([770, 1])
5 yl
torch.Size([770, 1])
torch.Size([4000, 1]) y
torch.Size([4000])
torch.Size([4000, 4]) x
5 p
torch.Size([770, 1])
5 yl
torch.Size([770, 1])
torch.Size([2000, 1]) y
torch.Size([2000])
torch.Size([2

KeyboardInterrupt: 

In [18]:
tr_loss

[[1, 1, 337.09033203125],
 [1, 2, 331.8153076171875],
 [1, 3, 292.12847900390625],
 [2, 1, 296.585205078125],
 [2, 2, 280.14599609375],
 [2, 3, 271.2704772949219],
 [3, 1, 250.09718322753906],
 [3, 2, 244.40707397460938],
 [3, 3, 227.0341796875],
 [4, 1, 220.47055053710938],
 [4, 2, 216.134765625],
 [4, 3, 200.00865173339844],
 [5, 1, 187.7088165283203],
 [5, 2, 172.44076538085938],
 [5, 3, 168.01290893554688],
 [6, 1, 163.15493774414062],
 [6, 2, 160.61769104003906],
 [6, 3, 171.02984619140625],
 [7, 1, 155.96754455566406],
 [7, 2, 157.88986206054688],
 [7, 3, 153.60079956054688],
 [8, 1, 142.91201782226562],
 [8, 2, 145.76634216308594],
 [8, 3, 152.1350860595703],
 [9, 1, 147.83865356445312],
 [9, 2, 143.379638671875],
 [9, 3, 146.55874633789062],
 [10, 1, 141.83526611328125],
 [10, 2, 136.55242919921875],
 [10, 3, 134.05552673339844],
 [11, 1, 132.486572265625],
 [11, 2, 137.46444702148438],
 [11, 3, 137.78561401367188],
 [12, 1, 134.34690856933594],
 [12, 2, 131.43739318847656],
 [

<font color='darkblue' size=7 > Sampler </font>

In [21]:
class LinearPosteriorSampling():
    """
    NN + linear
    """
    def __init__(self, num_features, num_actions, sigmultiplier= 'fixed', diagmat =False, scaled_down_error_variance= 0.1, R=0.5):

        #self.hparams       = hparams
        self.num_features  = num_features #number of nodes/output in L-1 layer of NN
        self.num_actions   = num_actions
        
        self.diagmat       = diagmat
        self.sigmultiplier = sigmultiplier

        self.scaled_down_error_variance = scaled_down_error_variance

        # No prior/ridge lambda, have set this to zero here.
        self._lambda_prior = 0.0001 #It can be set to higher value to bolster like a ridge regression 

        self.mu  = [ np.zeros(self.num_features)
                                        for _ in range(self.num_actions) ]

        self.cov = [(1.0 / self._lambda_prior) * np.eye(self.num_features)
                                        for _ in range(self.num_actions)]

        self.precision = [self._lambda_prior * np.eye(self.num_features)
                                        for _ in range(self.num_actions) ]
        
        self.R       = R
        self.epsilon = 0.1
        self.delta   = 0.5
        self.t       = 1
        
        self.sig_sq = self.R * np.sqrt((24 / self.epsilon) * self.num_features * np.log(self.t / self.delta))
        
        if self.sigmultiplier =='fixed':
             self.sig_sq_2 = [self.sig_sq  for _ in range(self.num_actions) ]
        elif (self.sigmultiplier =='unbiased_error_variance') or (self.sigmultiplier =='scaled_down_error_variance'):
             self.sig_sq_2 = [0.01  for _ in range(self.num_actions) ]

        self.t = 0
        self.data_cumulative = {}
        for i in range(self.num_actions):
            self.data_cumulative[i] = {'context': np.array([]) , 
                                        'reward': np.array([])}
            
        self.sig_sq_2_BKP_list = []

    def update_offline_data(self, context, action, reward):
        for i in range(self.num_actions):
            if len(self.data_cumulative[i]['context'])==0:
                self.data_cumulative[i]['context'] = copy.deepcopy(context[(action==i).ravel(),:]) 
                self.data_cumulative[i]['reward']  = copy.deepcopy(reward[(action==i).ravel(),:]) 
            else:
                #assert self.data_cumulative[i]['context'].shape==context.shape, 'Shape of numpy arrays of context/features does not match'
                self.data_cumulative[i]['context'] =  np.vstack((self.data_cumulative[i]['context'], context[(action==i).ravel(),:])) 
                self.data_cumulative[i]['reward']  =  np.vstack((self.data_cumulative[i]['reward'] , reward[(action==i).ravel(),:])) 
                
    def yourAPI_recommending_actions(self, mdlidx , context, size=1):
        if mdlidx ==1: #random
            if size==1:
                return np.random.choice(np.arange(self.num_actions), size=size)[0]
            else:
                return np.random.choice(np.arange(self.num_actions), size=size)
        elif mdlidx ==2: #Linear TS
            return self.recommend_action(context)

    def recommend_action(self, context):
        """Samples beta's from posterior, and chooses best action accordingly."""

        ## pass through NN
        mdl.eval()
        _, z = mdl.forward_custom(torch.from_numpy(context).float()[:,1:])
        try:
            if self.diagmat ==False:
                beta_s = [ np.random.multivariate_normal(self.mu[i], self.sig_sq_2[i] * self.cov[i]) 
                                         for i in range(self.num_actions) ]
            else:
                beta_s = [ np.random.multivariate_normal(self.mu[i], self.sig_sq_2[i] * (np.diag(self.cov[i]) * np.eye(self.num_features)) ) 
                                         for i in range(self.num_actions) ]
                
        except np.linalg.LinAlgError as e:
            # Sampling could fail if covariance matrix is not positive definite
            print('Exception when sampling for {}.'.format(self.name))
            print('Details: {} | {}.'.format(e.message, e.args))
            beta_s = [ np.random.multivariate_normal(np.zeros((self.num_features)), np.eye(self.num_features))
                                           for i in range(self.num_actions) ]

        # Apply Thompson Sampling principal to last-layer representation
        vals = [ np.dot(beta_s[i], z.T) for i in range(self.num_actions) ]
        return np.argmax(vals,axis=0)

    def update(self, context, action, reward):
        """Updates the posterior using linear bayesian regression formula."""

        #stack up new data
        self.t += 1
        self.update_offline_data(context, action, reward)

        # Update the Linear Regression (NOT a bayesian update, take full batch and update)
        for action_v in range(self.num_actions):
            # pull up full batch data from the offline training stack/batch
            z, y = self.data_cumulative[action_v]['context'], self.data_cumulative[action_v]['reward']  

            #pass through NN
            mdl.eval()
            _, z = mdl.forward_custom(torch.from_numpy(z).float()[:,1:])
            
            #OLS Regression updates
            s = np.dot(z.T, z)
            
            #print(f"iter - {self.t} , action_v - {action_v} , s matrix - {s}")
            precision_a = s + self._lambda_prior * np.eye(self.num_features)
            #precision_a = s 
            cov_a       = np.linalg.inv(precision_a)
            mu_a        = np.dot(cov_a, np.dot(z.T, y)).ravel()

            # Store parms
            self.mu[action_v]        = mu_a
            self.cov[action_v]       = cov_a
            self.precision[action_v] = precision_a

            #unbiased error variance
            y_hat                   = np.dot(z, mu_a).ravel()
            err                     = y.ravel() - y_hat
            sig_sq_2                = np.dot(err.T, err) /(z.shape[0] - len(mu_a.ravel())) #unbiased_error_variance

            if self.sigmultiplier =='fixed':
                pass #
            elif self.sigmultiplier =='unbiased_error_variance':
                self.sig_sq_2[action_v] = sig_sq_2
            elif self.sigmultiplier =='scaled_down_error_variance':
                self.sig_sq_2[action_v] = self.scaled_down_error_variance * sig_sq_2

            self.sig_sq_2_BKP_list.append([self.t, action_v, z.shape[0], self.sig_sq_2[action_v] , self.sig_sq_2[i] * np.diag(self.cov[i]) ])


In [20]:
class training_module():
    def __init__(self, sampled_data, num_features=4, num_actions=5, sigmultiplier= 'fixed', diagmat =False, scaled_down_error_variance= 0.1, R=0.5, knownfeatures=0):
        #
        self.lp             = LinearPosteriorSampling(num_features=4, num_actions=5, sigmultiplier= 'fixed', diagmat =False, scaled_down_error_variance= 0.1, R=0.5)
        self.sampled_data   = sampled_data
        self.num_features   = num_features
        self.ERROR_VARIANCE_STD = 0.25
        if isinstance(knownfeatures, np.ndarray ) or np.isscalar(knownfeatures):
            self.knownfeatures  = knownfeatures
        else:
            assert 0==1, 'Either pass a scalar in knownfeatures OR pass numpy array indices of columns that should be used in the feature set'

    def sampled_parm(self, act:int=0, size=1):
        # mu_vec, parmCovMat  
        return np.random.multivariate_normal(mean= mu_vec[act], cov= parmCovMat[act], size=size, check_valid='warn', tol=1e-8)

    def yourORG_as_a_blackBox(self, d, rec_actions):
        rewards_given_action = []
        for i, r in enumerate(d):
            parm    = self.sampled_parm(act=rec_actions[i], size=1)
            reward  = np.dot(parm , np.transpose(r)[..., np.newaxis]).ravel()
            err     = np.random.normal(loc=0, scale=self.ERROR_VARIANCE_STD, size=1)
            reward +=err
            rewards_given_action.append([np.asscalar(reward)])
        return np.array(rewards_given_action)
    
    def run_each_batch(self, btchsize, mdlidx=1):
        orgtrf_idx                    = np.random.choice(np.arange(len(self.sampled_data)), size= btchsize, replace=False,) #d[btchidx]
        if isinstance(self.knownfeatures, np.ndarray ):
             orgtraffic_known_features     = self.sampled_data[orgtrf_idx, :][:,self.knownfeatures]
        else:
             orgtraffic_known_features     = self.sampled_data[orgtrf_idx]
        orgtraffic_w_unknown_features = self.sampled_data[orgtrf_idx]
        rec_actions                   = self.lp.yourAPI_recommending_actions(mdlidx, context = orgtraffic_known_features, size=len(orgtraffic_known_features))

        rewards_seen_in_data          = self.yourORG_as_a_blackBox(orgtraffic_w_unknown_features, rec_actions)
        return orgtraffic_known_features, rec_actions[..., np.newaxis], rewards_seen_in_data

    def training_routine(self, btchsize, num_batches):
        batchnum  = 0
        #BATCHSIZE = btchsize
        d, a, r   = self.run_each_batch(btchsize= btchsize, mdlidx =1) # random selection of actions
            
        #update offline stacked data in lp object with new data, re-estimate regression coefficients (batch 0)
        self.lp.update(d, a, r)

        #batch 1 to N - explore exploit, update parameters
        reward_sum_list = [] 
        reward_sum_list.append([batchnum, r.ravel().mean(), r.ravel().sum()])
        for batchnum in range(1, num_batches): 
               #explore exploit
               d, a, r   = self.run_each_batch(btchsize= btchsize, mdlidx =2) 
               reward_sum_list.append([batchnum, r.ravel().mean(), r.ravel().sum()])
               #update offline stacked data in lp object with new data, re-estimate regression coefficients
               self.lp.update(d, a, r)

        dfres = pd.DataFrame(reward_sum_list, columns = ['batchnum','mean_reward','sum_reward'])

        variance_parms = pd.DataFrame(self.lp.sig_sq_2_BKP_list, columns=['iter','act','N', 'sig_square', 'sig_cov_prod' ])

        self.dfres          = dfres
        self.variance_parms = variance_parms

        #return dfres, variance_parms

In [22]:
#

In [23]:
%%time
allresults         = []
R                  = 0.5
scale_down_err_var = 1
NUM_TRIALS         = 5
knowf              = 0
for sm in ['scaled_down_error_variance']:
    for diagmat in [True, False]:
            for scale_down_err_var in [0.1, 0.25, 0.5, 0.8]:
                for trialnum in np.arange(1,NUM_TRIALS):
                    trn_m = training_module(sampled_data, 
                                           num_features  = mdl.layersizes[-1], #sampled_data.shape[1], 
                                           num_actions   = num_arms, 
                                           sigmultiplier = sm, 
                                           diagmat       = diagmat, 
                                           scaled_down_error_variance = scale_down_err_var, #relevant only for scaled_error_variance 
                                           R                          = 0.5, 
                                           knownfeatures              = 0)
                    trn_m.training_routine(btchsize=300, num_batches=100)
                    arm_dist                  = pd.DataFrame(np.expand_dims(trn_m.variance_parms.pivot(index='iter',columns='act', values=['N']).iloc[-1,:].values, axis=0), \
                                                                         columns=['arm_n_'+str(int(i)) for i in np.arange(num_arms)])
                    arm_dist['rew_mean']      = trn_m.dfres.iloc[-10:,:].mean_reward.mean()
                    arm_dist['rew_std' ]      = trn_m.dfres.iloc[-10:,:].mean_reward.std()
                    arm_dist['sigmultiplier'] = (sm if sm !='scaled_down_error_variance' else 'scldwn_errvar_'+ str(scale_down_err_var) ) + ("_diagmat" if diagmat==True else "_fullMat")
                    arm_dist['trialnum']      = trialnum
                    arm_dist['R']             = str(R)
                    arm_dist['scale_down_err_var'] = str(scale_down_err_var)
                    arm_dist['knownfeatures']      = str(knowf)
                    allresults.append(arm_dist)
for sm in ['fixed']:
    for diagmat in [True, False]:
            for R in [0.1, 0.2, 0.5]:
                for trialnum in np.arange(1,NUM_TRIALS):
                    trn_m = training_module(sampled_data, 
                                           num_features  = mdl.layersizes[-1], #sampled_data.shape[1], 
                                           num_actions   = num_arms, 
                                           sigmultiplier = sm, 
                                           diagmat       = diagmat, 
                                           scaled_down_error_variance = scale_down_err_var, #relevant only for scaled_error_variance 
                                           R                          = R, 
                                           knownfeatures              = 0)
                    trn_m.training_routine(btchsize=300, num_batches=100)
                    arm_dist                  = pd.DataFrame(np.expand_dims(trn_m.variance_parms.pivot(index='iter',columns='act', values=['N']).iloc[-1,:].values, axis=0), \
                                                                         columns=['arm_n_'+str(int(i)) for i in np.arange(num_arms)])
                    arm_dist['rew_mean']      = trn_m.dfres.iloc[-10:,:].mean_reward.mean()
                    arm_dist['rew_std' ]      = trn_m.dfres.iloc[-10:,:].mean_reward.std()
                    arm_dist['sigmultiplier'] = (sm if sm !='scaled_down_error_variance' else 'scldwn_errvar_'+ str(scale_down_err_var) ) + ("_diagmat" if diagmat==True else "_fullMat")
                    arm_dist['trialnum']      = trialnum
                    arm_dist['R']             = str(R)
                    arm_dist['scale_down_err_var'] = str(scale_down_err_var)
                    arm_dist['knownfeatures']      = str(knowf)
                    allresults.append(arm_dist)


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


CPU times: user 21min 52s, sys: 19.1 s, total: 22min 11s
Wall time: 5min 32s


In [148]:
allresults_bkp = copy.deepcopy(allresults)

In [24]:
allresults = pd.concat(allresults)
allresults.shape

(56, 12)

In [25]:
allresults['unq_series'] = allresults['sigmultiplier'] + '_'+ allresults['knownfeatures'] + '_R='+ allresults['R'].astype(str)
allresults

,arm_n_0,arm_n_1,arm_n_2,arm_n_3,arm_n_4,rew_mean,rew_std,sigmultiplier,trialnum,R,scale_down_err_var,knownfeatures,unq_series
0,418,142,1725,236,27479,7.70,0.30,scldwn_errvar_0.1_diagmat,1,0.5,0.1,0,scldwn_errvar_0.1_diagmat_0_R=0.5
0,375,288,2132,348,26857,7.93,0.33,scldwn_errvar_0.1_diagmat,2,0.5,0.1,0,scldwn_errvar_0.1_diagmat_0_R=0.5
0,254,318,1789,549,27090,8.00,0.29,scldwn_errvar_0.1_diagmat,3,0.5,0.1,0,scldwn_errvar_0.1_diagmat_0_R=0.5
0,980,435,1761,941,25883,7.73,0.32,scldwn_errvar_0.1_diagmat,4,0.5,0.1,0,scldwn_errvar_0.1_diagmat_0_R=0.5
0,389,333,1126,920,27232,7.87,0.26,scldwn_errvar_0.25_diagmat,1,0.5,0.25,0,scldwn_errvar_0.25_diagmat_0_R=0.5
0,225,195,1165,886,27529,7.78,0.31,scldwn_errvar_0.25_diagmat,2,0.5,0.25,0,scldwn_errvar_0.25_diagmat_0_R=0.5
0,359,340,1316,1143,26842,7.78,0.22,scldwn_errvar_0.25_diagmat,3,0.5,0.25,0,scldwn_errvar_0.25_diagmat_0_R=0.5
0,485,189,1202,1399,26725,7.82,0.24,scldwn_errvar_0.25_diagmat,4,0.5,0.25,0,scldwn_errvar_0.25_diagmat_0_R=0.5
0,1140,211,1813,471,26365,7.70,0.28,scldwn_errvar_0.5_diagmat,1,0.5,0.5,0,scldwn_errvar_0.5_diagmat_0_R=0.5
0,434,114,1175,1463,26814,7.68,0.25,scldwn_errvar_0.5_diagmat,2,0.5,0.5,0,scldwn_errvar_0.5_diagmat_0_R=0.5


In [26]:
allresults.unq_series.unique().tolist()

['scldwn_errvar_0.1_diagmat_0_R=0.5',
 'scldwn_errvar_0.25_diagmat_0_R=0.5',
 'scldwn_errvar_0.5_diagmat_0_R=0.5',
 'scldwn_errvar_0.8_diagmat_0_R=0.5',
 'scldwn_errvar_0.1_fullMat_0_R=0.5',
 'scldwn_errvar_0.25_fullMat_0_R=0.5',
 'scldwn_errvar_0.5_fullMat_0_R=0.5',
 'scldwn_errvar_0.8_fullMat_0_R=0.5',
 'fixed_diagmat_0_R=0.1',
 'fixed_diagmat_0_R=0.2',
 'fixed_diagmat_0_R=0.5',
 'fixed_fullMat_0_R=0.1',
 'fixed_fullMat_0_R=0.2',
 'fixed_fullMat_0_R=0.5']

In [27]:
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(25, 15))
# sns.lineplot(data=allresults, x='trialnum', y='rew_mean', hue='unq_series', style='unq_series', ax=ax, markers = True, ci=None)


In [28]:
dfagg = allresults.groupby('unq_series').agg({'rew_mean':['mean', 'median', 'std'], 'rew_std':['mean']}).reset_index()
dfagg.columns = [tpl[0] + '_' + tpl[1] for tpl in dfagg.columns]
dfagg.sort_values(by='rew_mean_mean', ascending=False)

,unq_series_,rew_mean_mean,rew_mean_median,rew_mean_std,rew_std_mean
1,fixed_diagmat_0_R=0.2,7.88,7.86,0.08,0.29
9,scldwn_errvar_0.25_fullMat_0_R=0.5,7.86,7.84,0.09,0.29
0,fixed_diagmat_0_R=0.1,7.85,7.86,0.12,0.18
6,scldwn_errvar_0.1_diagmat_0_R=0.5,7.84,7.83,0.15,0.31
2,fixed_diagmat_0_R=0.5,7.83,7.84,0.10,0.25
13,scldwn_errvar_0.8_fullMat_0_R=0.5,7.83,7.84,0.09,0.28
7,scldwn_errvar_0.1_fullMat_0_R=0.5,7.82,7.84,0.07,0.25
12,scldwn_errvar_0.8_diagmat_0_R=0.5,7.81,7.81,0.04,0.31
8,scldwn_errvar_0.25_diagmat_0_R=0.5,7.81,7.80,0.04,0.25
4,fixed_fullMat_0_R=0.2,7.81,7.79,0.06,0.29
